In [1]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define data directories
train_dir = '/kaggle/input/thai-handwritten-characters-recognition/train/train'

# Define image size
img_size = (128, 128)

# Define batch size
batch_size = 128

In [3]:
# Define image data generators
train_datagen = ImageDataGenerator(rescale=1/255., validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1/255.,validation_split=0.2)

# Generate training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    subset = "training",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Generate validation data
val_data = val_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    subset = "validation",
    batch_size=batch_size,
    class_mode='categorical'
)

Found 50687 images belonging to 68 classes.
Found 12640 images belonging to 68 classes.


# Base model

In [4]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(128,128,3),
    include_top=False,
    weights="imagenet",
)

9406464/9406464 [==============================] - 1s 0us/step


In [5]:
base_model.trainable = False

In [6]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [7]:
avg_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
pred_layer = tf.keras.layers.Dense(units=68, activation='softmax')(avg_pooling_layer)

In [8]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=pred_layer)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

# Train

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(train_data, epochs=25, validation_data=val_data)

Epoch 1/25
396/396 [==============================] - 263s 652ms/step - loss: 1.5544 - accuracy: 0.6064 - val_loss: 1.0576 - val_accuracy: 0.7041
Epoch 2/25
396/396 [==============================] - 84s 212ms/step - loss: 0.8343 - accuracy: 0.7693 - val_loss: 0.8690 - val_accuracy: 0.7458
Epoch 3/25
396/396 [==============================] - 84s 213ms/step - loss: 0.6895 - accuracy: 0.8045 - val_loss: 0.8003 - val_accuracy: 0.7610
Epoch 4/25
396/396 [==============================] - 84s 211ms/step - loss: 0.6073 - accuracy: 0.8245 - val_loss: 0.7649 - val_accuracy: 0.7687
Epoch 5/25
396/396 [==============================] - 83s 210ms/step - loss: 0.5520 - accuracy: 0.8394 - val_loss: 0.7475 - val_accuracy: 0.7748
Epoch 6/25
396/396 [==============================] - 87s 220ms/step - loss: 0.5103 - accuracy: 0.8519 - val_loss: 0.7188 - val_accuracy: 0.7812
Epoch 7/25
396/396 [==============================] - 91s 230ms/step - loss: 0.4763 - accuracy: 0.8612 - val_loss: 0.7161 - val_a

# Fine tuning

In [12]:
len(base_model.layers)

154

In [13]:
base_model.trainable = True

In [14]:
for layer in base_model.layers[:90]:
    layer.trainable = False

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
396/396 [==============================] - 96s 211ms/step - loss: 0.7090 - accuracy: 0.8300 - val_loss: 37.1437 - val_accuracy: 0.1212
Epoch 2/10
396/396 [==============================] - 87s 220ms/step - loss: 0.2664 - accuracy: 0.9158 - val_loss: 24.5649 - val_accuracy: 0.1996
Epoch 3/10
396/396 [==============================] - 83s 210ms/step - loss: 0.2086 - accuracy: 0.9332 - val_loss: 5.4054 - val_accuracy: 0.5342
Epoch 4/10
396/396 [==============================] - 88s 222ms/step - loss: 0.1780 - accuracy: 0.9427 - val_loss: 1.9980 - val_accuracy: 0.7170
Epoch 5/10
396/396 [==============================] - 88s 222ms/step - loss: 0.1640 - accuracy: 0.9455 - val_loss: 1.6293 - val_accuracy: 0.7479
Epoch 6/10
396/396 [==============================] - 85s 214ms/step - loss: 0.1507 - accuracy: 0.9506 - val_loss: 1.2054 - val_accuracy: 0.8013
Epoch 7/10
396/396 [==============================] - 95s 241ms/step - loss: 0.1436 - accuracy: 0.9534 - val_loss: 1.7236 - val_

# Submission

In [18]:
import pandas as pd

In [43]:
# Generate predictions using test-time augmentation 10 times
img_test = '/kaggle/input/thai-handwritten-characters-recognition/test'
sub_csv = '/kaggle/input/thai-handwritten-characters-recognition/submission.csv'
df_test = pd.read_csv(sub_csv)
df_test.head()

,filename,class
0,0.jpg,28.0
1,1.jpg,8.0
2,2.jpg,9.0
3,3.jpg,NaN
4,4.jpg,NaN


In [44]:
df_test['filename'] =df_test['filename'].apply(lambda x: '/kaggle/input/thai-handwritten-characters-recognition/test/test/'+x)
df_test['class'] = df_test['class'].astype(str)

In [45]:
df_test['filename'][0]

'/kaggle/input/thai-handwritten-characters-recognition/test/test/0.jpg'

In [46]:
test_generator = train_datagen.flow_from_dataframe(df_test, 
                                                    x_col='filename', 
                                                    y_col=None,
                                                    directory = img_test,
                                                    target_size=(128, 128),
                                                    class_mode=None,shuffle=False)

Found 13599 validated image filenames.


In [63]:
num_tta = 10
tta_preds = []
for i in range(num_tta):
    preds = model.predict(test_generator,3000)
    tta_preds.append(preds)

425/425 [==============================] - 17s 39ms/step


In [64]:
# Average the predictions obtained from the original image and its augmented versions
avg_preds = tf.reduce_mean(tta_preds, axis=0)

In [68]:
df_sub = pd.read_csv(sub_csv)

In [69]:
df = pd.DataFrame({'filename': df_sub['filename'], 'class': tf.argmax(avg_preds, axis=1).numpy()})
df.to_csv('predictions2.csv', index=False)